In [64]:
import pandas as pd
import numpy as np
import math
from random import sample

In [66]:
class DecisionTree:
    def __init__(self):
        self.split_feature = None
        self.split_value = None
        self.left = None
        self.right = None
        self.entropy = None
        self.prediction = None

    #this method grows the tree, we stop when information gain cannot pass threshold
    def fit(self, data: pd.DataFrame, labels: pd.Series, threshold: float) -> "DecisionTree":
        #initialize entropy as the entropy of the entered data
        self.entropy = DecisionTree.entropy(labels)
        gain = 0

        #if the entropy is 0 we just stop
        if self.entropy == 0:
            self.prediction = labels.iloc[0]
            return self
        
        #iterate over each feature, sort it if numerical, compute info gain of each splitting
        features = data.columns
        for feature in features:
        
            #first determine if its a categorical or numerical feature
            if data[feature].dtype == "object":
                #calculate best info gain splitting within feature
                features_best = DecisionTree.cat_info_gain(self.entropy, data, labels, feature)
                #record split if its biggest gain so far
                if (features_best[0] > gain):
                    gain = features_best[0]
                    self.split_feature = feature
                    self.split_value = features_best[1]

            #otherwise its a numerical split
            else:
                #calculate best info gain splitting within feature
                features_best = DecisionTree.num_info_gain(self.entropy, data, labels, feature)
                #record split if its biggest gain so far
                if (features_best[0] > gain):
                    gain = features_best[0]
                    self.split_feature = feature
                    self.split_value = features_best[1]

        #we have determined the splitting feature and value, now set children
        #if the split feature wasn't set then
        if self.split_feature is None:
            self.prediction = labels.mode().iloc[0]
            return self
            
        #seperate case for if splitting feature is categorical
        if data[self.split_feature].dtype == "object":
            left_data = data[data[self.split_feature] == self.split_value]
            left_labels = labels[data[self.split_feature] == self.split_value]
            right_data = data[data[self.split_feature] != self.split_value]
            right_labels = labels[data[self.split_feature] != self.split_value]

        else:
            left_data = data[data[self.split_feature] <= self.split_value]
            left_labels = labels[data[self.split_feature] <= self.split_value]
            right_data = data[data[self.split_feature] > self.split_value]
            right_labels = labels[data[self.split_feature] > self.split_value]

        #decide whether to stop or not
        if len(left_data) == 0 or len(right_data) == 0:
            # No meaningful split, stop recursion here
            self.prediction = labels.mode()[0]
            return self
        elif gain > threshold:
            #assign chilren
            self.left = DecisionTree().fit(left_data, left_labels, threshold)
            self.right = DecisionTree().fit(right_data, right_labels, threshold)
    
            return self

        else:
            self.prediction = labels.mode().iloc[0]
            return self
    
    def classify(self, data: pd.DataFrame) -> pd.Series:
        predictions = pd.Series(np.zeros(data.shape[0]), index=data.index)

        for index in data.index:
            current = self
            datum = data.loc[index]
            #traverse until leaf
            while current.prediction is None:
                to_bin = datum[current.split_feature]
                #check if categorical
                if isinstance(current.split_value, str):
                    if to_bin == current.split_value:
                        current = current.left
                    else:
                        current = current.right
                else:
                    if to_bin <= current.split_value:
                        current = current.left
                    else:
                        current = current.right
            
            #now at leaf, apply its prediction
            predictions[index] = current.prediction

        return predictions

    #calculates the entropy of one node
    @staticmethod
    def entropy(labels: pd.Series) -> float:
        total = len(labels)
        counts = labels.value_counts()

        total = len(labels)
        if total == 0:
            return 0
    
        counts = labels.value_counts()
        if len(counts) == 0:
            return 0
        
        p1 = counts.iloc[0] / total
        p2 = 1 - p1
        entropy_value = 0
        for p in [p1, p2]:
            if p > 0:
                entropy_value -= p * math.log(p)
        return entropy_value

    #calculating largest info gain split of a categorical feature
    @staticmethod
    def cat_info_gain(entropy: int, data: pd.DataFrame, labels: pd.Series, feature: str) -> list:
        categories = data[feature].unique()
        best_gain = 0
        best_value = None

        #compute info gain of splitting each category
        for category in categories:
            in_labels = labels[data[feature] == category]
            out_labels = labels[data[feature] != category]

            #record the split if its biggest gain so far
            split_gain = entropy - ((len(in_labels) * DecisionTree.entropy(in_labels) + len(out_labels) * DecisionTree.entropy(out_labels)) / len(labels))
            if (split_gain > best_gain):
                best_gain = split_gain
                best_value = category

        return [best_gain, best_value]

    #calculating largest info gain split of a numerical feature
    @staticmethod
    def num_info_gain(entropy: int, data: pd.DataFrame, labels: pd.Series, feature: str) -> list:
        #sort feature in ascending order, apply same to labels
        sorted_feat = data.sort_values(by = feature)[feature]
        sorted_labels = labels[sorted_feat.index]
        best_gain = 0
        best_value = None

        ones_in_bag = 0
        zeros_in_bag = 0
        ones_out_bag = len(labels[labels == 1])
        zeros_out_bag = len(labels[labels == 0])
        #traverse the data, updating bags and info gain at each split
        for i in range(len(sorted_feat)):
            if sorted_labels.iloc[i] == 1:
                ones_in_bag += 1
                ones_out_bag -= 1
            else:
                zeros_in_bag += 1
                zeros_out_bag -= 1
            #only update when the value changes (don't look at final value since that's 0 gain)
            if i != len(sorted_feat) - 1 and sorted_feat.iloc[i + 1] != sorted_feat.iloc[i]:
                split_gain = entropy - DecisionTree.weighted_entropy(ones_in_bag, ones_out_bag, zeros_in_bag, zeros_out_bag)
                if split_gain > best_gain:
                    best_gain = split_gain
                    best_value = sorted_feat.iloc[i]
        return [best_gain, best_value]

    #computes weighted entropy when give num 1's in bag, out bag, and the total num of labels (including 0's)
    @staticmethod
    def weighted_entropy(ones_in_bag: int, ones_out_bag: int, zeros_in_bag: int, zeros_out_bag: int) -> float:
        in_size = ones_in_bag + zeros_in_bag
        out_size = ones_out_bag + zeros_out_bag
    
        def safe_entropy(one, zero, size):
            if size == 0:
                return 0
            one_p = one / size
            zero_p = zero / size
            return -sum(p * math.log(p) for p in [one_p, zero_p] if p > 0)
    
        in_entropy = safe_entropy(ones_in_bag, zeros_in_bag, in_size)
        out_entropy = safe_entropy(ones_out_bag, zeros_out_bag, out_size)
    
        return (in_size * in_entropy + out_size * out_entropy) / (in_size + out_size)
        

In [68]:
tit_train = pd.read_csv("/Users/garrettbrown/Desktop/CS189/HWs/hw5_starter/titanic_train_cleaned.csv")
tit_test = pd.read_csv("/Users/garrettbrown/Desktop/CS189/HWs/hw5_starter/titanic_test_cleaned.csv")

In [70]:
#split into data and labels
tit_train_data = tit_train.loc[:, "pclass":]
tit_train_labels = tit_train.loc[:, "survived"]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
994    0.0
995    1.0
996    0.0
997    0.0
998    0.0
Name: survived, Length: 999, dtype: float64

In [72]:
#train model
clf = DecisionTree()
clf.fit(tit_train_data, tit_train_labels, .001)

In [74]:
#get predictions on the training data to verify that it makes sense
predictions = clf.classify(tit_train_data)
(predictions == tit_train_labels).sum() / len(predictions)

0.9419419419419419

In [76]:
#Now I want to use 5-fold cross-validation to tune the threshold parameter
rows = tit_train_data.shape[0]
indices = np.arange(rows)
np.random.shuffle(indices)
tit_train_data = tit_train_data.iloc[indices]
tit_train_labels = tit_train_labels.iloc[indices]


bins = [0, 200, 400, 600, 800, rows]
thresholds = [0, .0001, .001, .01, .05, .1, 1]
thresh_valid_means = np.empty(len(thresholds))
for j in range(len(thresholds)):
    valid_accuracy = np.empty(5)
    for i in range(len(bins) - 1):
        valid_data = tit_train_data.iloc[bins[i]:bins[i+1], :]
        lst = [tit_train_data.iloc[:bins[i], :], tit_train_data.iloc[bins[i+1]:, :]]
        train_data = pd.concat(lst)
        lst = [tit_train_labels.iloc[:bins[i]], tit_train_labels.iloc[bins[i+1]:]]
        train_labels = pd.concat(lst)
        valid_labels = tit_train_labels.iloc[bins[i]:bins[i+1]]
    
        clf = DecisionTree()
        clf.fit(train_data, train_labels, thresholds[j])
        predictions = clf.classify(valid_data)
        valid_accuracy[i] = (predictions == valid_labels).sum() / len(predictions)

    valid_accuracy = valid_accuracy.mean()
    thresh_valid_means[j] = valid_accuracy
    
best_thresh = thresholds[np.argmax(thresh_valid_means)]
print(best_thresh)

0.05


In [78]:
#Now train decision tree on the entire training data with best threshold
#Lower prediction accuracy possible since threshold may be lower and overfit less to training
clf = DecisionTree()
clf.fit(tit_train_data, tit_train_labels, best_thresh)
predictions = clf.classify(tit_train_data)
(predictions == tit_train_labels).sum() / len(predictions)

0.8068068068068068

In [80]:
#generate predictions for the training set
deterministic_predictions = clf.classify(tit_test)
deterministic_predictions.value_counts()

0.0    272
1.0    146
Name: count, dtype: int64

In [86]:
tit_test

,passengerid,pclass,sex,sibsp,parch,fare,cabin,embarked,age group
0,892,3,male,0,0,7.8292,N,Q,young adult
1,893,3,female,1,0,7.0000,N,S,adult
2,894,2,male,0,0,9.6875,N,Q,elderly
3,895,3,male,0,0,8.6625,N,S,young adult
4,896,3,female,1,1,12.2875,N,S,young adult
...,...,...,...,...,...,...,...,...,...
413,1305,3,male,0,0,8.0500,N,S,N
414,1306,1,female,0,0,108.9000,C,C,adult
415,1307,3,male,0,0,7.2500,N,S,adult
416,1308,3,male,0,0,8.0500,N,S,N


In [88]:
#make submission file in competition format
deterministic_predictions = pd.DataFrame(deterministic_predictions)
deterministic_predictions = deterministic_predictions.rename(columns = {0: "Survived"})
deterministic_predictions["PassengerId"] = tit_test["passengerid"]
deterministic_predictions["Survived"] = deterministic_predictions["Survived"].astype(int)
deterministic_predictions = deterministic_predictions[["PassengerId", "Survived"]]
deterministic_predictions

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [56]:
#My first submission the model trained to have .05 threshold
#I scored .78 accuracy in the Kaggle competition with that
deterministic_predictions.to_csv('/Users/garrettbrown/Desktop/CS189/HWs/hw5_starter/titanic_det_submission.csv', index=False)

In [108]:
#Now we train a random forest, following general advice of square root total feats
num_trees = 1000
num_features = 4
num_data = tit_train_data.shape[0]
features = list(tit_train_data.columns)
forest = []

for i in range(num_trees):
    random_feats = sample(features, 3)
    bag = np.random.choice(num_data, size = num_data, replace = True)
    clf = DecisionTree()
    clf.fit(tit_train_data.loc[bag, random_feats], tit_train_labels.loc[bag], best_thresh)
    forest.append(clf)

In [110]:
#look at accuracy on the training data
predictions = pd.Series(np.zeros(tit_train_data.shape[0])).astype(float)

for tree in forest:
    predictions += tree.classify(tit_train_data)

predictions = predictions / num_trees
predictions = (predictions > .5).astype(int)
train_accuracy = (predictions.reset_index(drop=True) == tit_train_labels.reset_index(drop=True)).sum() / len(predictions)
train_accuracy

0.5735735735735735

In [112]:
#now get predictions on the test set
predictions = pd.Series(np.zeros(tit_test.shape[0])).astype(float)

for tree in forest:
    predictions += tree.classify(tit_test)

predictions = predictions / num_trees
random_predictions = (predictions > .5).astype(int)

In [114]:
#make submission file in competition format
random_predictions = pd.DataFrame(random_predictions)
random_predictions = random_predictions.rename(columns = {0: "Survived"})
random_predictions["PassengerId"] = tit_test["passengerid"]
random_predictions["Survived"] = random_predictions["Survived"].astype(int)
random_predictions = random_predictions[["PassengerId", "Survived"]]
random_predictions

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [116]:
#Now I submit the random forest predictions
#I scored .81 accuracy in the Kaggle competition with that
#that's shocking to me since the performance on the training data
#was not that good.
random_predictions.to_csv('/Users/garrettbrown/Desktop/CS189/HWs/hw5_starter/titanic_rand_submission.csv', index=False)